In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense, Dropout,Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from keras import initializers
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_recall_fscore_support,accuracy_score

In [46]:
df= pd.read_csv('/content/drive/MyDrive/WISDM.csv').drop('Unnamed: 0', axis = 1)


Jogging = df[df['activity']=='Jogging'].head(4500).copy()
Sitting = df[df['activity']=='Sitting'].head(4500).copy()

balanced_data = pd.DataFrame()

balanced_data = pd.concat([balanced_data, Jogging,  Sitting])

balanced_data.shape


(9000, 6)

In [47]:
X_train, X_test = train_test_split(balanced_data, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((7200, 6), (1800, 6))

In [48]:
X_train = X_train[X_train.activity == 'Jogging']
X_train = X_train.drop('activity' , axis =1)

y_test = X_test.activity
X_test = X_test.drop('activity', axis =1)

In [49]:

sc = MinMaxScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [50]:
NUM_COL = 5
#Setting the Hyperparameters

#learning_rate = 0.00001
learning_rate = 0.0001
batch_size = 512
epochs = 5
adam = Adam(learning_rate = learning_rate, beta_1 = 0.5)

In [51]:
def get_generator(optimizer):

    generator = Sequential()
    generator.add(Dense(64, input_dim=5, kernel_initializer=initializers.glorot_normal(seed=42)))
    generator.add(Activation('tanh'))

    generator.add(Dense(128))
    generator.add(Activation('tanh'))

    generator.add(Dense(256))
    generator.add(Activation('tanh'))

    generator.add(Dense(256))
    generator.add(Activation('tanh'))

    generator.add(Dense(512))
    generator.add(Activation('tanh'))

    generator.add(Dense(5, activation='tanh'))

    generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

    return generator

In [52]:
def get_discriminator(optimizer):

    discriminator = Sequential()

    discriminator.add(Dense(256, input_dim=NUM_COL, kernel_initializer=initializers.glorot_normal(seed=42)))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(128))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(128))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(128))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(128))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(1))
    discriminator.add(Activation('sigmoid'))

    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

    return discriminator

In [53]:
def get_gan_network(discriminator, generator, optimizer,input_dim=NUM_COL):

    discriminator.trainable = False
    gan_input = Input(shape=(input_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))


    return gan

In [54]:

batch_count = X_train.shape[0] // batch_size
gan_loss = []
discriminator_loss = []

generator = get_generator(adam)
discriminator = get_discriminator(adam)
gan = get_gan_network(discriminator, generator, adam,input_dim=NUM_COL)


In [55]:
for epoch in range(epochs):
    for index in tqdm(range(batch_count)):
        # Creating a random set of input noise and images
        noise = np.random.normal(0, 1, size=[batch_size,NUM_COL])

        # Generate fake samples
        generated_images = generator.predict_on_batch(noise)

        #Obtain a batch of normal network packets
        image_batch = X_train[index * batch_size: (index + 1) * batch_size]

        X = np.vstack((generated_images,image_batch))
        y_dis = np.ones(2*batch_size)
        y_dis[:batch_size] = 0

        # Train discriminator
        discriminator.trainable = True
        d_loss= discriminator.train_on_batch(X, y_dis)

        # Train generator
        noise = np.random.uniform(0, 1, size=[batch_size, NUM_COL])
        y_gen = np.ones(batch_size)
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gen)

        #Record the losses
        discriminator_loss.append(d_loss)
        gan_loss.append(g_loss)

    print("Epoch %d Batch %d/%d [D loss: %f] [G loss:%f]" % (epoch,index,batch_count, d_loss, g_loss))

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 0 Batch 6/7 [D loss: 0.624820] [G loss:0.544564]


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1 Batch 6/7 [D loss: 0.620719] [G loss:0.660460]


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 2 Batch 6/7 [D loss: 0.023353] [G loss:6.093113]


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 3 Batch 6/7 [D loss: 0.007715] [G loss:9.416176]


  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 4 Batch 6/7 [D loss: 0.003003] [G loss:10.657322]


In [56]:
nr_batches_test = np.ceil(X_test.shape[0] // batch_size).astype(np.int32)

nr_batches_test

3

In [57]:
results =[]

for t in tqdm(range(nr_batches_test +1)):
        ran_from = t * batch_size
        ran_to = (t + 1) * batch_size
        image_batch = X_test[ran_from:ran_to]
        tmp_rslt = discriminator.predict(x=image_batch,batch_size=128,verbose=0)
        results = np.append(results, tmp_rslt)

  0%|          | 0/4 [00:00<?, ?it/s]

In [60]:
y_test.value_counts()

Sitting    916
Jogging    884
Name: activity, dtype: int64

In [63]:
y2 = y_test.to_numpy()

pd.options.display.float_format = '{:20,.7f}'.format
results_df = pd.concat([pd.DataFrame(results),pd.DataFrame(y2)], axis=1)
results_df.columns = ['results','y_test']
print ('Mean score for Normal data :', results_df.loc[results_df['y_test'] == 'Sitting', 'results'].mean() )
print ('Mean score for Fall data :', results_df.loc[results_df['y_test'] == 'Jogging', 'results'].mean())

Mean score for Normal data : 0.9969416740940127
Mean score for Fall data : 0.9995488646478136
